# Exercise: Heterogenous Plugin and the DevCloud

In this exercise, we will load a model using the hetero plugin on to the FPGA and CPU, and the GPU and CPU. We will then perform an inference on it and compare the time it takes to do the same for each device pair.

<span class="graffiti-highlight graffiti-id_z8bfs11-id_d97ox8f"><i></i><button>Graffiti Sample Button (edit me)</button></span>



#### Set up paths so we can run Dev Cloud utilities
You *must* run this every time they enter a Workspace session.

In [1]:
%env PATH=/opt/conda/bin:/opt/spark-2.4.3-bin-hadoop2.7/bin:/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/opt/intel_devcloud_support
import os
import sys
sys.path.insert(0, os.path.abspath('/opt/intel_devcloud_support'))
sys.path.insert(0, os.path.abspath('/opt/intel'))

env: PATH=/opt/conda/bin:/opt/spark-2.4.3-bin-hadoop2.7/bin:/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/opt/intel_devcloud_support


## The model

We will be using the `vehicle-license-plate-detection-barrier-0106` model for this exercise. Remember that to run a model using the HETERO Plugin, we need to use FP16 as the model precision.

The model is present in the `/data/models/intel` folder.

# Step 1: Creating a Python Script

The first step is to create a python script that you can use to load the model and perform an inference. I have used the `writefile` magic to create a python file called `inference_on_device.py`. You will need to complete this file.

In [47]:
%%writefile inference_on_device.py

import time
import numpy as np
import cv2
from openvino.inference_engine import IENetwork
from openvino.inference_engine import IECore
import argparse

def main(args):
    model=args.model_path
    model_weights=model+'.bin'
    model_structure=model+'.xml'
    
    start=time.time()
    print('Started')
    
    # TODO: Load the model on VPU
    model=IENetwork(model_structure, model_weights)
    print('Model loaded')

    core = IECore()
    print('Core created')
    
    net = core.load_network(network=model, device_name=args.device, num_requests=1)
    print('Network loaded')

    print(f"Time taken to load model = {time.time()-start} seconds")
    
    # Reading and Preprocessing Image
    input_img=cv2.imread('/data/resources/car.png')
    input_img=cv2.resize(input_img, (300,300), interpolation = cv2.INTER_AREA)
    input_img=np.moveaxis(input_img, -1, 0)
    print('Image prepared')

    # TODO: Prepare the model for inference (create input dict etc.)
    input_name=next(iter(model.inputs))
    input_dict={input_name:input_img}
    print('Input created')
    
    start=time.time()
    for _ in range(10):
        # TODO: Run Inference in a Loop
        net.infer(input_dict)
        print('Inference performed')
    
    print(f"Time Taken to run 10 Infernce using HETERO plugin is = {time.time()-start} seconds")

if __name__=='__main__':
    parser=argparse.ArgumentParser()
    parser.add_argument('--model_path', required=True)
    parser.add_argument('--device', default=None)
    
    args=parser.parse_args() 
    main(args)

Overwriting inference_on_device.py


<span class="graffiti-highlight graffiti-id_1rnmf5g-id_nmeqj1a"><i></i><button>Show Solution</button></span>

## Step 2: Creating a job submission script

To submit a job to the devcloud, we need to create a script. I have named the script as `inference_hetero_model_job.sh`.

Can you write a script that will take the model path and device as a command line argument and then call the python file you created in the previous cell with the path to the model?

In [48]:
%%writefile inference_fpga_model_job.sh
#!/bin/bash

exec 1>/output/stdout.log 2>/output/stderr.log

mkdir -p /output

DEVICE=$1
MODELPATH=$2


source /opt/intel/init_openvino.sh
aocl program acl0 /opt/intel/openvino/bitstreams/a10_vision_design_sg1_bitstreams/2019R4_PL1_FP16_MobileNet_Clamp.aocx


# Run the load model python script
python3 inference_on_device.py  --model_path ${MODELPATH} --device ${DEVICE}

cd /output

tar zcvf output.tgz stdout.log stderr.log

Overwriting inference_fpga_model_job.sh


<span class="graffiti-highlight graffiti-id_f1nbmn9-id_ia7yjlq"><i></i><button>Show Solution</button></span>

## Step 3a: Running on the FPGA

In the cell below, can you write the qsub command that will submit your job to the CPU?

In [28]:
fpga_job = !qsub inference_fpga_model_job.sh -d . -l nodes=1:tank-870:i5-6500te:iei-mustang-f100-a10 -F "HETERO:FPGA,CPU /data/models/intel/vehicle-license-plate-detection-barrier-0106/FP32/vehicle-license-plate-detection-barrier-0106" -N store_core 
print(fpga_job[0])

B8MXnXsIQ0PAFpRaO4LaRcI9Knp3nXTq


<span class="graffiti-highlight graffiti-id_cvp3lyi-id_chmeh50"><i></i><button>Show Solution</button></span>

## Step 3b: Running on GPU

In [49]:
gpu_job = !qsub inference_fpga_model_job.sh -d . -l nodes=s002-n011:i5-6500te:intel-hd-530 -F "HETERO:GPU,CPU /data/models/intel/vehicle-license-plate-detection-barrier-0106/FP32/vehicle-license-plate-detection-barrier-0106" -N store_core 
print(gpu_job[0])

E8Au22grFc6XKlo7Pm4U7wxsltq3veVh


<span class="graffiti-highlight graffiti-id_7k34s6u-id_022l4bj"><i></i><button>Show Solution</button></span>

## Step 4: Getting the Live Stat Values

By running the below command, we can see the live status of the commands.

<span class="graffiti-highlight graffiti-id_clj7fxa-id_d3gqjz0"><i></i><button>Graffiti Sample Button (edit me)</button></span>

In [50]:
import liveQStat
liveQStat.liveQStat()

## Step 5a: Get the results for FPGA

Running the cell below will get the output files from our job

<span class="graffiti-highlight graffiti-id_cygruth-id_6nd1x96"><i></i><button>Graffiti Sample Button (edit me)</button></span>

In [30]:
import get_results

get_results.getResults(fpga_job[0], filename="output.tgz", blocking=True)

getResults() is blocking until results of the job (id:B8MXnXsIQ0PAFpRaO4LaRcI9Knp3nXTq) are ready.
Please wait..............Success!
output.tgz was downloaded in the same folder as this notebook.


In [31]:
!tar zxf output.tgz

In [32]:
!cat stdout.log

Started
Model loaded
Core created


In [33]:
!cat stderr.log

./submission.sh: line 11: /opt/intel/init_openvino.sh: No such file or directory
./submission.sh: line 12: aocl: command not found
Traceback (most recent call last):
  File "inference_on_device.py", line 51, in <module>
    main(args)
  File "inference_on_device.py", line 24, in main
    net = core.load_network(network=model, device_name=args.device, num_requests=1)
  File "ie_api.pyx", line 134, in openvino.inference_engine.ie_api.IECore.load_network
  File "ie_api.pyx", line 141, in openvino.inference_engine.ie_api.IECore.load_network
RuntimeError: Failed to create plugin libdliaPlugin.so for device FPGA
Please, check your environment
Cannot load library 'libdliaPlugin.so': libdliaPlugin.so: cannot open shared object file: No such file or directory

tar: stdout.log: file changed as we read it


## Step 5b: Get the result for GPU

In [51]:
import get_results

get_results.getResults(gpu_job[0], filename="output.tgz", blocking=True)

getResults() is blocking until results of the job (id:E8Au22grFc6XKlo7Pm4U7wxsltq3veVh) are ready.
Please wait..........................................Success!
output.tgz was downloaded in the same folder as this notebook.


In [52]:
!tar zxf output.tgz

In [53]:
!cat stdout.log

DetectionOutput_Reshape_priors_/Output_0/Data__const is CPU
DetectionOutput_Reshape_conf_ is CPU
SSD/concat_reshape_softmax/mbox_conf_final is CPU
SSD/concat_reshape_softmax/Reshape is GPU
SSD/concat_reshape_softmax/mbox_conf_logits is GPU
SSD/ssd_head_1/Flatten_1/flatten/Reshape is GPU
SSD/ssd_head_1/layer_18/output_mbox_conf/BiasAdd/Add/Transpose is GPU
DetectionOutput_Reshape_loc_ is CPU
SSD/concat_reshape_softmax/mbox_loc_final is CPU
SSD/ssd_head_1/Flatten/flatten/Reshape is GPU
SSD/ssd_head_1/layer_18/output_mbox_loc/BiasAdd/Add/Transpose is GPU
SSD/ssd_head_2/Flatten_1/flatten/Reshape is GPU
SSD/ssd_head_2/feature_map_1_mbox_conf/BiasAdd/Add/Transpose is GPU
SSD/ssd_head_2/Flatten/flatten/Reshape is GPU
SSD/ssd_head_2/feature_map_1_mbox_loc/BiasAdd/Add/Transpose is GPU
SSD/ssd_head_3/Flatten_1/flatten/Reshape is GPU
SSD/ssd_head_3/feature_map_2_mbox_conf/BiasAdd/Add/Transpose is GPU
SSD/ssd_head_3/Flatten/flatten/Reshape is GPU
SSD/ssd_head_3/feature_map_2_mbox_loc/BiasAdd/Add/T

In [54]:
!cat stderr.log

./submission.sh: line 11: /opt/intel/init_openvino.sh: No such file or directory
./submission.sh: line 12: aocl: command not found
tar: stdout.log: file changed as we read it
